<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/cohesion_test/%5Bdistiluse_base_multilingual_cased_v1%5Dsentence_transformer_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence transformers 로딩

In [74]:
pip install -U sentence-transformers datasets

In [75]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [76]:
datasets = load_dataset("klue", "sts")

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


  0%|          | 0/2 [00:00<?, ?it/s]

# KLUE STS dataset
https://github.com/KLUE-benchmark/KLUE/wiki/KLUE-STS-dataset-description

In [77]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [78]:
import pandas as pd
train_samples = []
for data in datasets['train']:
  s1 = data['sentence1']
  s2 = data['sentence2']
  score = data['labels']['binary-label']
  train_samples.append({'sentence1':s1, 'sentence2': s2, 'label':score})
train_df = pd.DataFrame(train_samples)

In [79]:
train_df.head()

,sentence1,sentence2,label
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1


In [80]:
import numpy as np

train_df['cos_sim'] = np.nan # 컬럼 생성 및 초기화
train_df['abs_diff'] = np.nan # 컬럼 생성 및 초기화

# 모델 로드

In [81]:
import torch
from sentence_transformers import SentenceTransformer, util

model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'


embedding_model = models.Transformer(model_name)

pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooler])


Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/514M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 문장 유사도 평가 및 차이 추출

In [82]:
s1 = 'sentence1'
s2 = 'sentence2'

for i, row in train_df[:1000].iterrows():
  emb1 = model.encode(row[s1])
  emb2 = model.encode(row[s2])
  cos_score = abs(util.pytorch_cos_sim(emb1, emb2)[0])

  train_df['cos_sim'][i] = round(cos_score.item(), 2)
  train_df['abs_diff'][i] = abs(row['label'] - round(cos_score.item(), 2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [83]:
train_df[:20]

,sentence1,sentence2,label,cos_sim,abs_diff
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,1,0.92,0.08
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0,0.10,0.10
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0,0.37,0.37
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0,0.38,0.38
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,1,0.96,0.04
5,정부가 새로운 일자리를 직접 창출하는 노력도 배가하겠습니다.,세계에서 우리만큼 오랜 역사와 문화를 공유하는 가까운 이웃이 없습니다.,0,0.03,0.03
6,지하철을 타도 30분안에는 이동이 가능합니다!,"지하철을 탄다고 해도, 30분이면 그곳에 도착할 수 있어요!",1,0.90,0.10
7,사례집은 국립환경과학원 누리집(ecolibrary.me.go.kr)에서 12일부터 ...,주말을 제외한 평일 오후 12시 30분부터 문예회관 공식 페이스북과 유튜브에서는 지...,0,0.24,0.24
8,환퐁기 작동 방법 좀 설명해줘,조명등 낮에 켜놓으면 큰일나,0,0.07,0.07
9,새로운 친구들을 만나고 싶을때 아주 추천합니다.,새로운 친구들을 만나고 싶을 때 추천합니다.,1,0.98,0.02


# 오차 평균 값 계산

In [84]:
diff_total = train_df['abs_diff'][:1000].sum()
print(f'{model_name}의 오차 평균값 : ',diff_total/1000)

sentence-transformers/distiluse-base-multilingual-cased-v1의 오차 평균값 :  0.23183999999999996
